# CS506 Project Initial Report

## Chengyu Deng, Xiaotong Niu, Qian Zhang

This Jupyter Notebook is the code for the CS506 Initial Report. We processed the main data set and make it ready to use for analysis. 


**Part 1 Data processing**

We used Building Energy Reporting And Disclosure Ordinance (BERDO) data provided by [Analyze Boston - Boston.gov](https://data.boston.gov/) to do the analysis. 
* **_berdo2017.csv_** for the year of 2017.
* **_2016-reported-energy-and-water-metrics.xlsx_** for the year of 2016.
* **_2015-reported-energy-and-water-metrics.xlsx_** for the year of 2015.

The source of the data is: https://data.boston.gov/dataset/building-energy-reporting-and-disclosure-ordinance


In [39]:
# Import data and put them into Pandas dataframes
import csv
import pandas as pd
import numpy as np
from sklearn.preprocessing import normalize


# 2015 xlsx -> csv
data_2015_xlsx = pd.read_excel('2015-reported-energy-and-water-metrics.xlsx', index_col=None)
data_2015_xlsx.to_csv('berdo2015.csv')

# 2016 xlsx -> csv
data_2016_xlsx = pd.read_excel('2016-reported-energy-and-water-metrics.xlsx', index_col=None)
data_2016_xlsx.to_csv('berdo2016.csv')

# 2017
df_2015 = pd.read_csv('berdo2015.csv')
df_2016 = pd.read_csv('berdo2016.csv')
df_2017 = pd.read_csv('berdo2017.csv', encoding = "ISO-8859-1")


# Testing code (For debugging purpose)

# print(type(df_2015))
# print(type(df_2016))
# print(type(df_2017))
# print('-------------------------------')
# print(df_2015.shape)
# print(df_2016.shape)
# print(df_2017.shape)
# print('-------------------------------')
# print(list(df_2015.columns.values))
# print('-------------------------------')
# print(list(df_2016.columns.values))
# print('-------------------------------')
# print(list(df_2017.columns.values))
# print('-------------------------------')

In [2]:
# Data Trimming

# Dataframe columns names handling
if 'Years Reported' in df_2016:
    df_2016.drop('Years Reported', axis = 1, inplace = True)
if 'Years Reported' in df_2017:
    df_2017.drop('Years Reported', axis = 1, inplace = True)

if ' Gross Area (sq ft) ' in df_2017:
    df_2017.rename(index=str, columns={' Gross Area (sq ft) ': 'Gross Area (sq ft)'}, inplace = True)
if ' GHG Emissions (MTCO2e) ' in df_2017:
    df_2017.rename(index=str, columns={' GHG Emissions (MTCO2e) ': 'GHG Emissions (MTCO2e)'}, inplace = True)
if ' Total Site Energy (kBTU) ' in df_2017:
    df_2017.rename(index=str, columns={' Total Site Energy (kBTU) ': 'Total Site Energy (kBTU)'}, inplace = True)
if ' Onsite Renewable (kWh) ' in df_2017:
    df_2017.rename(index=str, columns={' Onsite Renewable (kWh) ': 'Onsite Renewable (kWh)'}, inplace = True)

    
# Testing code (For debugging purpose)

# print(type(df_2015))
# print(type(df_2016))
# print(type(df_2017))
# print(df_2015.shape)
# print(df_2016.shape)
# print(df_2017.shape)
# print('-------------------------------')
# print(list(df_2015.columns.values))
# print('-------------------------------')
# print(list(df_2016.columns.values))
# print('-------------------------------')
# print(list(df_2017.columns.values))
# print('-------------------------------')


# Select properties which belong to BU
with open("BU_Property_List.csv", 'r') as propertyFile:
    reader = csv.reader(propertyFile, delimiter='\t')
    propertyList_convol = list(reader)
    
    propertyList = []
    for each in propertyList_convol:
        for element in each:
            propertyList.append(element)
            
df2015_BU = df_2015.loc[df_2015['Property Name'].isin(propertyList)]
df2016_BU = df_2016.loc[df_2016['Property Name'].isin(propertyList)]
df2017_BU = df_2017.loc[df_2017['Property Name'].isin(propertyList)]

# Testing code (For debugging purpose)

print('BU properties ------------------------------------')
print(df2015_BU.shape)
print(df2016_BU.shape)
print(df2017_BU.shape)

# Drop rows of data whose Site EUI (kBTU/sf) is not available

df2015_BU = df2015_BU.loc[df_2015['Site EUI (kBTU/sf)'] != 'Not Available']
df2016_BU = df2016_BU.loc[df_2016['Site EUI (kBTU/sf)'] != 'Not Available']
df2017_BU = df2017_BU.loc[df_2017['Site EUI (kBTU/sf)'] != 'Not Available']

# Testing code (For debugging purpose)

# print(len(propertyList))
# print('------------------------------------')
# print(propertyList)
# print('------------------------------------')
# # print(df2015_BU)
# # print(df2016_BU)
# # print(df2017_BU)
print('BU properties Site EUI (kBTU/sf)] != Not Available----')
print(df2015_BU.shape)
print(df2016_BU.shape)
print(df2017_BU.shape)
print('------------------------------------')
print(list(df_2015.columns.values))
print('------------------------------------')
# print(df2015_BU['Property Name'])
# print(df2016_BU['Property Name'])
# print(df2017_BU['Property Name'])
# print('------------------------------------')
# print(df2017_BU[['Site EUI (kBTU/sf)', '% Electricity', '% Gas', '% Steam']])

BU properties ------------------------------------
(55, 23)
(56, 23)
(59, 23)
BU properties Site EUI (kBTU/sf)] != Not Available----
(37, 23)
(54, 23)
(57, 23)
------------------------------------
['Unnamed: 0', 'Property Name', 'Reported', 'Property Type', 'Address', 'ZIP', 'Gross Area (sq ft)', 'Site EUI (kBTU/sf)', 'Energy Star Score', 'Energy Star Certified', 'Property Uses', 'Year Built', 'GHG Emissions (MTCO2e)', 'GHG Intensity (kgCO2/sf)', 'Total Site Energy (kBTU)', '% Electricity', '% Gas', '% Steam', 'Water Intensity (gal/sf)', 'Onsite Solar (kWh)', 'User Submitted Info', 'User Submitted Link', 'Tax Parcel']
------------------------------------


**Part 2 Monthly energy consumption assignment**

Since the BERDO data is yearly based, we need to calculate the monthly energy consumption for each building based on the energy consumption distribution provided by Kevin Zheng from Sustainability@BU

In [67]:
# Calculate monthly energy consumption electricity/natrual gas

#[Jan, Feb, Mar, ..., Nov, Dec]
share_2015 = np.array([0.2038, 0.1062, 0.0778, 0.0620, 0.0589, 0.0588, 0.0473, 0.0813, 0.0705, 0.0732, 0.0673, 0.0929])
share_2016 = np.array([0.0919, 0.1179, 0.0995, 0.0642, 0.0710, 0.0733, 0.0660, 0.0673, 0.0733, 0.0774, 0.0959, 0.1023])
share_2017 = np.array([0.1146, 0.1031, 0.0872, 0.0862, 0.0699, 0.0650, 0.0661, 0.0635, 0.0865, 0.0671, 0.0727, 0.1181])

#          Jan                               Dec
#[[Natural Gas, Electricity], ...[Natural Gas, Electricity]]
NESplits_2015 = np.array(
    [[0.6877, 0.3123], 
     [0.6654, 0.3346], 
     [0.6079, 0.3921], 
     [0.5086, 0.4914], 
     [0.3888, 0.6112], 
     [0.3663, 0.6337], 
     [0.1913, 0.8087], 
     [0.5173, 0.4827], 
     [0.5090, 0.4910], 
     [0.5598, 0.4402], 
     [0.5245, 0.4755], 
     [0.6743, 0.3257]])

NESplits_2016 = np.array(
    [[0.6519, 0.3481], 
     [0.6802, 0.3198], 
     [0.6496, 0.3504], 
     [0.5395, 0.4605], 
     [0.4350, 0.5650], 
     [0.4154, 0.5846], 
     [0.3339, 0.6661], 
     [0.3336, 0.6664], 
     [0.4442, 0.5558], 
     [0.5321, 0.4679], 
     [0.6225, 0.3775], 
     [0.6477, 0.3523]])

NESplits_2017 = np.array(
    [[0.6739, 0.3261], 
     [0.6707, 0.3293], 
     [0.6211, 0.3789], 
     [0.5920, 0.4080], 
     [0.5074, 0.4926], 
     [0.3905, 0.6095], 
     [0.3199, 0.6801], 
     [0.3178, 0.6822],
     [0.3672, 0.6328], 
     [0.3960, 0.6040],  
     [0.5386, 0.4614], 
     [0.6525, 0.3475]])


# Testing code (For debugging purpose)
# print(np.sum(share_2015))
# print(np.sum(share_2016))
# print(np.sum(share_2017))
# print('------------------------------------')
# print(np.sum(NESplits_2015))
# print(np.sum(NESplits_2016))
# print(np.sum(NESplits_2017))
# print('------------------------------------')
# print(share_2015.shape)
# print(share_2016.shape)
# print(share_2017.shape)

share_2015 = np.reshape(share_2015, (share_2015.shape[0], 1))
share_2016 = np.reshape(share_2016, (share_2016.shape[0], 1))
share_2017 = np.reshape(share_2017, (share_2017.shape[0], 1))


# Testing code (For debugging purpose)

# print('------------------------------------')
# print(share_2015.shape)
# print(share_2016.shape)
# print(share_2017.shape)
# print('------------------------------------')
# print(NESplits_2015.shape)
# print(NESplits_2016.shape)
# print(NESplits_2017.shape)

share_2015_G_E = share_2015 * NESplits_2015
share_2016_G_E = share_2016 * NESplits_2016
share_2017_G_E = share_2017 * NESplits_2017


# Testing code (For debugging purpose)

# print('------------------------------------')
# print(share_2015_G_E.shape)
# print(share_2016_G_E.shape)
# print(share_2017_G_E.shape)
# print(share_2015_G_E)
# print(share_2016_G_E)
# print(share_2017_G_E)
# print('------------------------------------')

# print(np.sum(share_2015_G_E, axis = 1))
# print(np.sum(share_2016_G_E, axis = 1))
# print(np.sum(share_2017_G_E, axis = 1))

# Normalize the seperated distribution of electircity/natrual gas comsumption
G_dstri_2015 = share_2015_G_E[:, 0]
E_dstri_2015 = share_2015_G_E[:, 1]

G_dstri_2016 = share_2016_G_E[:, 0]
E_dstri_2016 = share_2016_G_E[:, 1]

G_dstri_2017 = share_2017_G_E[:, 0]
E_dstri_2017 = share_2017_G_E[:, 1]

# Reshape from (12, ) to (12, 1)
G_dstri_2015 = np.reshape(G_dstri_2015, (G_dstri_2015.shape[0], 1))
E_dstri_2015 = np.reshape(E_dstri_2015, (E_dstri_2015.shape[0], 1))

G_dstri_2016 = np.reshape(G_dstri_2016, (G_dstri_2016.shape[0], 1))
E_dstri_2016 = np.reshape(E_dstri_2016, (E_dstri_2016.shape[0], 1))

G_dstri_2017 = np.reshape(G_dstri_2017, (G_dstri_2017.shape[0], 1))
E_dstri_2017 = np.reshape(E_dstri_2017, (E_dstri_2017.shape[0], 1))

# Testing code (For debugging purpose)

# print('------------------------------------')
# print(share_2015_G_E)
# print(G_dstri_2015)
# print(E_dstri_2015)

# print('------------------------------------')
# print(share_2016_G_E)
# print(G_dstri_2016)
# print(E_dstri_2016)

# print('------------------------------------')
# print(share_2017_G_E)
# print(G_dstri_2017)
# print(E_dstri_2017)

G_dstri_2015_Norm = normalize(G_dstri_2015, norm='l1', axis = 0)
E_dstri_2015_Norm = normalize(E_dstri_2015, norm='l1', axis = 0)

G_dstri_2016_Norm = normalize(G_dstri_2016, norm='l1', axis = 0)
E_dstri_2016_Norm = normalize(E_dstri_2016, norm='l1', axis = 0)

G_dstri_2017_Norm = normalize(G_dstri_2017, norm='l1', axis = 0)
E_dstri_2017_Norm = normalize(E_dstri_2017, norm='l1', axis = 0)

# print(G_dstri_2015_Norm)
# print(E_dstri_2015_Norm)
# print(G_dstri_2016_Norm)
# print(E_dstri_2016_Norm)
# print(G_dstri_2017_Norm)
# print(E_dstri_2017_Norm)
# print('------------------------------------')

# print(np.sum(G_dstri_2015_Norm))
# print(np.sum(E_dstri_2015_Norm))
# print(np.sum(G_dstri_2016_Norm))
# print(np.sum(E_dstri_2016_Norm))
# print(np.sum(G_dstri_2017_Norm))
# print(np.sum(E_dstri_2017_Norm))


In [68]:
# One more data trim... 
'''
Since the distribution only involve with gas and electricity, we CURRENTLY drop the buildings that use steam. 
However in the future, if we get the revised consumption distribution over electricity, gas and steam, we will
re-include those dropped buildings that use steam back to analysis. 
'''

df2015_BU = df2015_BU.loc[df2015_BU['% Steam'] == '0%']
df2016_BU = df2016_BU.loc[df2016_BU['% Steam'].isnull()]
df2017_BU = df2017_BU.loc[df2017_BU['% Steam'].isnull()]

# Testing code (For debugging purpose)
# print(df2015_BU[['Site EUI (kBTU/sf)', '% Electricity', '% Gas', '% Steam']])
# print('----------------------------')
# print(df2016_BU[['Site EUI (kBTU/sf)', '% Electricity', '% Gas', '% Steam']])
# print('----------------------------')
# print(df2017_BU[['Site EUI (kBTU/sf)', '% Electricity', '% Gas', '% Steam']])
# print('----------------------------')

In [ ]:
# Assignment montly data based on the split



**Part 3 Visualization of monthly energy consumption**

Visualize the processed data and be ready to do further analysis. 

In [ ]:
# Data visualization

# Visualize:
# G_dstri_2015_Norm 
# E_dstri_2015_Norm 

# G_dstri_2016_Norm 
# E_dstri_2016_Norm 

# G_dstri_2017_Norm 
# E_dstri_2017_Norm 
